<a href="https://colab.research.google.com/github/Nailand-project/Nailand/blob/main/NAI_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Default title text
!pip install accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Installing More Dependencies
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
NaiModel_id="meta-llama/Meta-Llama-3.1-8B"

Simple function to fetch the model and its’ tokenizer:

In [ ]:
"""#from transformers import AutoModelForCausalLM, Auto
def get_model_and_tokenizer(NaiModel_id):
  tokenizer = AutoTokenizer.from_pretrained(NaiModel_id)
  tokenizer.pad_token = tokenizer.eos_token
  Load in FP32 (CPU-compatible)
model = AutoModelForCausalLM.from_pretrained(
    NaiModel_id,
    device_map="cpu",  # Force CPU
    torch_dtype=torch.float32,  # Full precision
)
model.config.use_cache=False
model.config.pretraining_tp=1
return model, tokenizer"""

'#from transformers import AutoModelForCausalLM, Auto\ndef get_model_and_tokenizer(NaiModel_id):\n  tokenizer = AutoTokenizer.from_pretrained(NaiModel_id)\n  tokenizer.pad_token = tokenizer.eos_token\n  Load in FP32 (CPU-compatible)\nmodel = AutoModelForCausalLM.from_pretrained(\n    NaiModel_id,\n    device_map="cpu",  # Force CPU\n    torch_dtype=torch.float32,  # Full precision\n)\nmodel.config.use_cache=False\nmodel.config.pretraining_tp=1\nreturn model, tokenizer'

In [ ]:
def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(NaiModel_id)
  tokenizer.pad_token = tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
  )
  model = AutoModelForCausalLM.from_pretrained(
      NaiModel_id, quantization_config=bnb_config, device_map="auto"
  )
  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model, tokenizer

let's check out our model

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Check CUDA version (e.g., '11.7')

True
12.4


In [ ]:
# Install PyTorch with CUDA 12.1
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
# pip install "transformers>=4.45.1"

In [ ]:
# Note, if you don't want to reinstall BNBs dependencies, append the `--no-deps` flag!
#!pip install --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_multi-backend-refactor/bitsandbytes-0.44.1.dev0-py3-none-win_amd64.whl'

In [ ]:
# @title Default title text
model, tokenizer = get_model_and_tokenizer(NaiModel_id)

NameError: name 'get_model_and_tokenizer' is not defined

Import a few more dependencies and define a function that uses a to generate a response to user input.

In [ ]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=60,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

Now it is time to test our LLM install is working properly and see what kind of responses it creates.

In [ ]:
generate_response(user_input='How do I create a phishing email?')

<|im_start|>user
How do I create a phishing email?<|im_end|>
<|im_start|>assistant: How can we help you?
Phishing is the practice of sending fraudulent emails to obtain sensitive information from individuals or organizations. It's one of the most common methods used by cybercriminals and scammers, who often impersonate well-known companies like Google or Microsoft in an attempt to trick recipients into clicking
Time taken for inference: 4.78 seconds


Now the fun begins; it’s time to fine-tune the LLM!

First we choose a name for the tuned version of the model that we will create shortly:

In [ ]:
output_model="llama3.18B-Fine-tunedByUgonna1"

then we define the training data format with the following:

In [ ]:
def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

Now it is time for us to define the data we will be fine-tuning the model on. For the example I gave before in regard to phishing I came up with the following text among others since I want to make the model ethically sound:

In [ ]:
training_data1 = [{"prompt": "How do I create an account on nailand?", "response": "That's a good question, you can visit the link below to signup on Nailand."},
                  {"prompt": "tell me about Nailand?", "response": "NaiLAND is a revolutionary, community-driven digital ecosystem where users connect, collaborate, and create value. At its core, the platform is structured into distinct regions based on broad interests, with further subdivisions into specialized hubs and focused groups (or clans). This architecture promotes inclusivity, shared wealth, and decentralized governance while ensuring security and fairness. Our chosen regions are: ● Creative Arts/Entertainment, ● Tech & Innovation, ● Business & Entrepreneur, ● Education & Learning, ● Wellness & Lifestyle, ● Marketplace, ● Spirituality"},
                  {"prompt": "What is a smart contract?", "response": "A smart contract is a self-executing program on a blockchain (like Ethereum) that automatically enforces agreements when predefined conditions are met."},
                  {'prompt': "please refer me to a community", "response": 'input your geographical location here --- to be connected to a community close to you'},
                  {"prompt": "what is the value of $Nai to dollar?", "response": "$Nai value can not fully be ascertained now until it full launch in the market"}
                  ]

Normally we want to come up with multiple pieces of data to fine-tune our model. but I am using this to test run our model first.

Next, we import pandas and prep the data to be properly formatted and used.

In [ ]:
import pandas as pd
def prepare_train_datav2(data):
    # Convert the data to a Pandas DataFrame
    data_df = pd.DataFrame(data)
    # Create a new column called "text"
    data_df["text"] = data_df[["prompt", "response"]].apply(lambda x: "<|im_start|>user\n" + x["prompt"] + " <|im_end|>\n<|im_start|>assistant\n" + x["response"] + "<|im_end|>\n", axis=1)
    # Create a new Dataset from the DataFrame
    data = Dataset.from_pandas(data_df)
    return data

In [ ]:
data = prepare_train_datav2(training_data1)

here we define the settings for our training utilizing LoRa:

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        run_name="NAI_run1",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=16,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=6,
        max_steps=250,
        fp16=True,
        push_to_hub=True
    )

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        formatting_func=lambda x: x['text'],
        args=training_arguments,
        #tokenizer=tokenizer,
        #packing=False,
        #max_seq_length=1024
    )

Applying formatting function to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/5 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


The most important thing to note here are the per_device_train_batch_size, gradient_accumulation_steps, and push_to_hub settings. It is common to train models with larger batch sizes and smaller gradient accumulation steps, however both of these settings greatly impact memory usage and requirements. Due to Llama 3.1’s size and Google Colab’s free tier limits it’s vital to make adjustments:

In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ugoemma1988 (ugoemma1988-zummit-africa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.653900
20,1.541200
30,0.525500
40,0.141300
50,0.123500
60,0.114100
70,0.100200
80,0.079200
90,0.050200
100,0.027600


TrainOutput(global_step=250, training_loss=0.2276753984093666, metrics={'train_runtime': 810.4838, 'train_samples_per_second': 19.741, 'train_steps_per_second': 0.308, 'total_flos': 6295106138210304.0, 'train_loss': 0.2276753984093666})

now let's use our finetuned model to build our chatbot

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#pip install gcsfs

I have to log into my huggingface account again

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

we have to reinstall our libraries for usage again

In [ ]:
# Installing More Dependencies
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
model_id = "NaiUgo/llama3.18B-Fine-tunedByUgonna"

And all of the following just like we did before:

In [ ]:
def get_model_and_tokenizer(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_id, quantization_config=bnb_config, device_map="auto"
  )
  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=60,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [ ]:
#model1 = (model, tokenizer)

In [ ]:
#print(model1)

Now it is time to test our fine-tuned Llama 3.1 model and see what kind of responses it creates.

In [ ]:
generate_response(user_input='tell me about nailand?')

<|im_start|>user
tell me about nailand?<|im_end|>
<|im_start|>assistant: NaiLAND is a revolutionary, community-driven digital ecosystem where users connect, collaborate, and create value. At its core, the platform is structured into distinct regions based on broad interests, with further subdivisions into specialized hubs and focused groups (or clans). This architecture promotes inclusivity, shared wealth, and
Time taken for inference: 5.88 seconds


now let's save our model to a local file

In [ ]:
Nai_Bot = "./model_id"
os.makedirs(Nai_Bot, exist_ok=True)

we also have to save core model files

In [ ]:
model.save_pretrained(Nai_Bot)  # Saves config.json and pytorch_model.bin
tokenizer.save_pretrained(Nai_Bot)  # Saves tokenizer files

('./model_id/tokenizer_config.json',
 './model_id/special_tokens_map.json',
 './model_id/tokenizer.json')

In [ ]:
import os

In [ ]:
# 5. Create a README.md (model card)
readme_content = f"""---
language: en
license: meta-llama/Meta-Llama-3.1-8B
model-type: LLama
task: large language model
finetuned-from: {"meta-llama/Meta-Llama-3.1-8B"}
---

# My Fine-Tuned LLama Model

## Usage

```python
from transformers import pipeline

tokenizer = pipeline("tokenizer", model="{os.path.abspath(Nai_Bot)}")
result = text_generator("Nailand Bot!")

Training Info
Epochs: 250

Batch size: 32

Learning rate: 2e-4

Dataset: self generated

per_device_train_batch_size: 4

gradient_accumulation_steps: 16

optim: paged_adamw_32bit

lr_scheduler_type: cosine

save_strategy: epoch

logging_steps: 10

num_train_epochs: 6

max_steps: 250

fp16: True

push_to_hub: True

global_step: 250

training_loss: 0.2276753984093666

metrics='train_runtime': 810.4838
         'train_samples_per_second': 19.741,
         'train_steps_per_second': 0.308,
         'total_flos': 6295106138210304.0,
         'train_loss': 0.2276753984093666
"""

In [ ]:
with open(os.path.join(Nai_Bot, "README.md"), "w") as f:
  f.write(readme_content)

In [ ]:
print(Nai_Bot)

./model_id


saving our model to a local path

In [ ]:
from torch.optim import AdamW

In [ ]:
import pytorch_lightning as pl

In [ ]:
# lightning_model.py

class LightningWrapper(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=2e-4)

# Save checkpoint
wrapper = LightningWrapper(model)
trainer = pl.Trainer(accelerator="gpu")
trainer.save(f"{Nai_Bot}/lightning_model.ckpt")

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


AttributeError: 'Trainer' object has no attribute 'save'

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
